# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Get the file paths with the subdirectories using glob
file_path_list = glob.glob(os.path.join(filepath,'*.csv'))
#print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

    # Read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Create a CSV reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # Extract each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Uncomment the code below to print the total number of rows 
print(len(full_data_rows_list))

# Uncomment the code below to print the contents of event data rows
#print(full_data_rows_list)

# Creating a smaller event data CSV file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# Print the number of rows in the CSV file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Connect to a Cassandra instance in local machine 
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# Connect to a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a keyspace to store the tables
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as exep:
    print(exep)

#### Set Keyspace

In [7]:
# Set the keyspace to sparkify
try:
    session.set_keyspace('sparkify')
except Exception as exep:
    print(exep)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Cassandra Data Modeling

### Query 1 - Retrieve the artist, song title and song's length in the music app history that was heard during a given session ID and itemInSession

In [8]:
# A table specific to the query has to be created 
# Here the PRIMARY KEY is chosen to be ((sessionId), item_in_session) and PARTITION KEY to be sessionId
# We are querying on these two attributes and sessionId will be the better choice as PARTITION KEY as it will be hashed to find the cluster
# Since we need to query on itemInSession, it is chosen to be a clustering column and is added in the PRIMARY KEY 

# Create table
query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(session_id INT, item_in_session INT, artist_name VARCHAR, song_title text, song_duration DECIMAL, PRIMARY KEY ((session_id), item_in_session))"

try:
    session.execute(query)
except Exception as exep:
    print(exep)

#### Insert data into the tables

In [9]:
# Load the data from the CSV file and insert into the Cassandra table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        # Insert into the table
        query = "INSERT INTO music_library (session_id, item_in_session, artist_name, song_title, song_duration) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        # Properly assign the values from CSV to the table attributes
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as exep:
            print(exep)

#### Verifing that the data have been inserted into table

In [10]:
# SELECT statement to verify whether the data was properly entered into the table
select_query = "SELECT artist_name, song_title, song_duration FROM music_library WHERE session_id=338 AND item_in_session=4"

try:
    rows = session.execute(select_query)
except Exception as exep:
    print(exep)

# Print the results
for row in rows:
    print(row.artist_name, row.song_title, row.song_duration)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Query 2 - Retrieve the name of artist, song (sorted by itemInSession) and user name (first and last name) given an user ID and session ID

In [11]:
# A table specific to the query has to be created 
# Here the PRIMARY KEY is chosen to be ((user_id, sessionId), item_in_session) and PARTITION KEY to be (user_id, sessionId)
# We are querying on these two attributes and we want the results to be sorted in order of itemInSession
# If only userId was used as PARTITION KEY, then there exists a high chance of data aggregating in a single node (depends on data)
# Hence using (user_id, sessionId) as PARTITION KEY, it is more likely to be distributed across nodes as sessionId changes periodically
# Since the output needs to be sorted in itemInSession, it is chosen to be a clustering column and is added in the PRIMARY KEY 

# Create table
query = "CREATE TABLE IF NOT EXISTS artist_library "
query = query + "(user_id INT, session_id INT, item_in_session INT, artist_name VARCHAR, song_title TEXT, first_name VARCHAR, last_name VARCHAR, \
                    PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as exep:
    print(exep)

#### Insert data into the tables

In [12]:
# Load the data from the CSV file and insert into the Cassandra table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        # Insert into the table
        query = "INSERT INTO artist_library (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Properly assign the values from CSV to the table attributes
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as exep:
            print(exep)

#### Verifing that the data have been inserted into table

In [13]:
# SELECT statement to verify whether the data was properly entered into the table
select_query = "SELECT artist_name, song_title, first_name, last_name FROM artist_library WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(select_query)
except Exception as exep:
    print(exep)

# Print the results
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3 - Retrieve the user name (first and last) in the music app history who listened to a particular song

In [14]:
# A table specific to the query has to be created 
# Here the PRIMARY KEY is chosen to be ((song_title), userId) and PARTITION KEY to be song_title
# We are querying on the song_title and hence it should be the PARTITION KEY
# If only song_title was as PRIMARY KEY, since Cassandra does not allow duplicate entries, the entries will be overwritten
# Hence using (song_title, userId) as PRIMARY KEY, we can overcome this issue without violating the query constraints
# Since the output needs to be sorted in itemInSession, it is chosen to be a clustering column and is added in the PRIMARY KEY 

# Create table
query = "CREATE TABLE IF NOT EXISTS play_history "
query = query + "(song_title TEXT, user_id INT, first_name VARCHAR, last_name VARCHAR, PRIMARY KEY ((song_title), user_id))"

try:
    session.execute(query)
except Exception as exep:
    print(exep)

#### Insert data into the tables

In [15]:
# Load the data from the CSV file and insert into the Cassandra table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        # Insert into the table
        query = "INSERT INTO play_history (song_title, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        
        # Properly assign the values from CSV to the table attributes
        try:
            session.execute(query, (line[1], int(line[10]), line[4], line[9]))
        except Exception as exep:
            print(exep)

#### Verifing that the data have been inserted into table

In [16]:
# SELECT statement to verify whether the data was properly entered into the table
select_query = "SELECT first_name, last_name FROM play_history WHERE song_title='All Hands Against His Own'"

try:
    rows = session.execute(select_query)
except Exception as exep:
    print(exep)

# Print the results
for row in rows:
    print(row.artist_name, row.song_title, row.song_duration)

### Drop the tables before closing out the sessions

In [17]:
try:
    session.execute("DROP TABLE IF EXISTS music_library")
except Execption as exep:
    print(exep)
    
try:
    session.execute("DROP TABLE IF EXISTS artist_library")
except Execption as exep:
    print(exep)
    
try:
    session.execute("DROP TABLE IF EXISTS play_history")
except Execption as exep:
    print(exep)  

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()